> 장르 속성을 이용한 영화 콘텐츠 기반 필터링

<br>

: 콘텐츠 기반 필터링은 사용자가 특정 영화를 감상하고 그 영화를 좋아했다면 그 영화와 비슷한 특성/속성, 구성요소 등을 가진 다른 영화를 추천하는 것입니다

ex) 영화 인셉션을 재미있게 봤다면 인셉션의 장르인 액션, 공상과학으로 높은 평점을 받은 다른 영화를 추천하거나 인셉션의 감독인 크리스토퍼 놀란의 다른 영화를 추천하는 방식입니다 

: 영화(또는 상품/서비스) 간의 유사성을 판단하는 기준이 영화를 구성하는 다양한 콘텐츠 (장르, 감독, 배우, 평점, 키워드, 영화 설명)를 기반으로 하는 방식이 바로 콘텐츠 기반 필터링

> 데이터 로딩 및 가공 

In [104]:
import pandas as pd
import numpy as np
import warnings ; warnings.filterwarnings("ignore")

In [105]:
movies=pd.read_csv("/Users/ijiseon/Desktop/ESAA-OB/archive/tmdb_5000_movies.csv")

In [106]:
print(movies.shape)

(4803, 20)


In [107]:
movies.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""name"": ""Fantasy""}, {...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"": 2964, ""name"": ""future""}, {""id"": 3386, ""name"": ""sp...",en,Avatar,"In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, ...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289}, {""name"": ""Twentieth Century Fox Film Corporatio...","[{""iso_3166_1"": ""US"", ""name"": ""United States of America""}, {""iso_3166_1"": ""GB"", ""name"": ""United ...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso_639_1"": ""es"", ""name"": ""Espa\u00f1ol""}]",Released,Enter the World of Pandora.,Avatar,7.2,11800


- genres, keywords 등과 같은 칼럼을 보면 파이썬 리스트 내부에 여러 개의 딕셔너리가 있는 형태의 문자열로 표기돼 있음. 이는 한꺼번에 여러 개의 값을 표현하기 위한 표기 방식 

  예를 들어 아바타의 장르는 액션과 어드벤처 등의 여러가지 장르로 구성될 수 있기 때문
  
  ->하지만 이 칼럼이 DataFrame으로 만들어질 때는 단순히 문자열 형태로 로딩되므로 이 칼럼을 가공하지 않고는 필요한 정보를 추출할 수가 없음

In [108]:
movies_df = movies[["id","title","genres","vote_average","vote_count","popularity",
                   "keywords","overview"]]

In [109]:
pd.set_option("max_colwidth",100)
movies_df[["genres","keywords"]][:1]

,genres,keywords
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""name"": ""Fantasy""}, {...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"": 2964, ""name"": ""future""}, {""id"": 3386, ""name"": ""sp..."


- 위와 같이 genres 칼럼은 여러 개의 개별 장르 데이터를 가지고 있고, 이 개별 장르의 명칭은 딕셔너리의 키인 "name"으로 추출할 수 있음. Keywords 역시 마찬가지 구조를 가지고 있음

- genres 칼럼의 문자열을 분해해서 개별 장르를 파이썬 리스트 객체로 추출하겠습니다 

   -> **파이썬 ast 모듈의 litera_eval()함수 이용**
   : 문자열이 의미하는 list[dict1,dict2] 객체로 만들 수 있음

In [110]:
from ast import literal_eval

movies_df["genres"] = movies_df["genres"].apply(literal_eval)
movies_df["keywords"]=movies_df["keywords"].apply(literal_eval)

In [111]:
pd.set_option("max_colwidth",100)
movies_df[["genres","keywords"]][:1]

,genres,keywords
0,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'name': 'Adventure'}, {'id': 14, 'name': 'Fantasy'}, {...","[{'id': 1463, 'name': 'culture clash'}, {'id': 2964, 'name': 'future'}, {'id': 3386, 'name': 'sp..."


- 이제 genres 칼럼은 문자열이 아니라 실제 내부에서 여러 장르 딕셔너리로 구성된 객체를 가집니다. 이제 정말 장르명만 리스트 객체로 추출

In [112]:
movies_df["genres"]=movies_df["genres"].apply(lambda x : [y["name"] for y in x])
movies_df["keywords"]=movies_df["keywords"].apply(lambda x : [y["name"] for y in x])
movies_df[["genres","keywords"]][:1]

,genres,keywords
0,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colony, society, space travel, futuristic, romance, spa..."


> 장르 콘텐츠 유사도 측정

: genres 칼럼은 여러 개의 개별 장르가 리스트로 구성돼 있음. 

-> 가장 간단한 방법은 genres를 문자열로 변경한 뒤 이를 CountVectorizer로 피처벡터화한 행렬 데이터 값을 코사인 유사도로 비교하는 것입니다. 

1. 문자열로 변환된 genres 칼럼을 Count 기반으로 피처 벡터화 변환 
<br>
2. genres 문자열을 피처 벡터화 행렬로 변환한 데이터 세트를 코사인 유사도를 통해 비교합니다. 이를 위해 데이터 세트의 레코드별로 타 레코드와 장르에서 코사인 유사도 값을 가지는 객체를 생성 
<br>
3. 장르 유사도가 높은 영화 중에 평점이 높은 순으로 영화를 추천 

In [113]:
from sklearn.feature_extraction.text import CountVectorizer

# CountVectorizer를 적용하기 위해 공백문자로 word 단위가 구분되는 문자열로 변환
movies_df["genres_literal"] = movies_df["genres"].apply(lambda x : (" ").join(x))
count_vect = CountVectorizer(min_df=1, ngram_range=(1,2))
genre_mat = count_vect.fit_transform(movies_df["genres_literal"])
print(genre_mat.shape)

(4803, 276)


In [114]:
from sklearn.feature_extraction.text import CountVectorizer

# CountVectorizer를 적용하기 위해 공백문자로 word 단위가 구분되는 문자열로 변환
movies_df["genres_literal"] = movies_df["genres"].apply(lambda x : (" ").join(x))
count_vect = CountVectorizer(min_df=0.00001, ngram_range=(1,2))
genre_mat = count_vect.fit_transform(movies_df["genres_literal"])
print(genre_mat.shape)

(4803, 276)


In [115]:
movies_df["genres_literal"]

0       Action Adventure Fantasy Science Fiction
1                       Adventure Fantasy Action
2                         Action Adventure Crime
3                    Action Crime Drama Thriller
4               Action Adventure Science Fiction
                          ...                   
4798                       Action Crime Thriller
4799                              Comedy Romance
4800               Comedy Drama Romance TV Movie
4801                                            
4802                                 Documentary
Name: genres_literal, Length: 4803, dtype: object

- CountVectorizer의 min_df 파라미터는 0보다 큰 정수 또는 0.0보다 크고 1.0 이하의 부동 소수점이어야 합니다. min_df 파라미터는 어휘에 포함되기 위해 특정 단어가 나타나야 하는 최소 문서 수를 지정합니다. min_df=0으로 설정하면, 이론적으로는 모든 단어가 포함될 수 있지만, 실제로는 이 값이 유효하지 않습니다.

- min_df=1이면 모든 단어가 최소 한 번 이상 나타난 경우에 어휘에 포함됩니다.
- min_df=0.1과 같이 부동 소수점으로 설정하면 전체 문서의 10% 이상에 나타나는 단어만 어휘에 포함됩니다.

min_df=0 설정은 Scikit-learn 라이브러리에서 유효하지 않으므로, 코드 실행 시 오류가 발생합니다.

------------------------------------------------------------------------

In [116]:
from sklearn.metrics.pairwise import cosine_similarity

genre_sim = cosine_similarity(genre_mat, genre_mat)
print(genre_sim.shape)
print(genre_sim[:1])

(4803, 4803)
[[1.         0.59628479 0.4472136  ... 0.         0.         0.        ]]


- cosine_similarities() 호출로 생성된 genre_sim 객체는 movies_df의 genre_literal 칼럼을 피처 벡터화한 행렬 (genre_mat) 데이터 행별 유사도 정보를 가지고 있음 = movies_df DataFrame의 행별 장르 유사도 값을 가지고 있는 것 

- movies_df를 장르 기준으로 콘텐츠 기반 필터링을 수행하려면 movies_df의 개별 레코드에 대해서 갖아 장르 유사도가 높은 순으로 다른 레코드를 추출해야 하는데, 이를 위해 앞에서 생성한 genre_sim 객체를 이용 

- genre_sim.argsort()[:, ::-1]

  : genre_sim 객체의 기준 행별로 비교 대상이 되는 행의 유사도 값이 높은 순으로 정렬된 행령의 위치 인덱스 값을 추출하면 됨. 값이 높은 순으로 정렬된 비교 대상 행의 유사도 값이 아니라 비교대상 행의 위치 인덱스임에 주의! 
 

In [117]:
genre_sim_sorted_ind = genre_sim.argsort()[:, ::-1]

In [118]:
print(genre_sim_sorted_ind[:1])

[[  14  813 3494 ...   77 4801 4775]]


-> genre_sim.argsort()[:, ::-1]를 사용해 높은 순으로 정렬된 비교 행 위치 인덱스 값을 가져오고 그중에 0번 레코드의 비교 행 위치 인덱스 값만 샘플로 추출

> 장르 콘텐츠 필터링을 이용한 영화 추천 

In [119]:
def find_sim_movie(df, sorted_ind, title_name, top_n=10):
    # 인자로 입력된 movies_df DataFrame에서 title 칼럼이 입력된 title_name값인 DataFrame 추출
    title_movie = df[df["title"]==title_name]
    
    #title_named을 가진 DataFrame의 index 객체를 ndarray로 반환하고
    #sorted_ind 인자로 입력된 genre_sim_sorted_ind 객체에서 유사도 순으로 top_n개의 index 추출
    title_index = title_movie.index.values
    similar_indexes = sorted_ind[title_index, :(top_n)]
    
    #추출된 top_n index 출력, top_n index는 2차원 데이터임
    #dataframe에서 index로 사용하기 위해서 1차원 array로 변경
    print(similar_indexes)
    similar_indexes = similar_indexes.reshape(-1)
    
    return df.iloc[similar_indexes]

In [120]:
similar_movies = find_sim_movie(movies_df, genre_sim_sorted_ind, "The Godfather", 10)
similar_movies[["title","vote_average"]]

[[ 281 3378 3594 1464 1946 1370 4065 2839 2731 4217]]


,title,vote_average
281,American Gangster,7.4
3378,Auto Focus,6.1
3594,Spring Breakers,5.0
1464,Black Water Transit,0.0
1946,The Bad Lieutenant: Port of Call - New Orleans,6.0
1370,21,6.5
4065,Mi America,0.0
2839,Rounders,6.9
2731,The Godfather: Part II,8.3
4217,Kids,6.8


In [121]:
movies_df[["title","vote_average","vote_count"]].sort_values("vote_average",ascending=False)[:10]

,title,vote_average,vote_count
4662,Little Big Top,10.0,1
4247,Me You and Five Bucks,10.0,2
4045,"Dancer, Texas Pop. 81",10.0,1
3519,Stiff Upper Lips,10.0,1
3992,Sardaarji,9.5,2
2386,One Man's Hero,9.3,2
2970,There Goes My Baby,8.5,2
1881,The Shawshank Redemption,8.5,8205
3337,The Godfather,8.4,5893
2796,The Prisoner of Zenda,8.4,11


In [122]:
C = movies_df["vote_average"].mean()
m = movies_df["vote_count"].quantile(0.6)
print("C:", round(C,3), "m:",round(m,3))

C: 6.092 m: 370.2


In [123]:
percentile = 0.6
m = movies["vote_count"].quantile(percentile)
C = movies["vote_average"].mean()

In [124]:
def weighted_vote_average(record):
    v = record["vote_count"]
    R = record["vote_average"]
    
    return ( (v/(v+m)*R) +(m/(m+v))*C )

In [125]:
movies_df["weighted_vote"]=movies_df.apply(weighted_vote_average,axis=1)

In [126]:
movies_df[["title","vote_average","weighted_vote","vote_count"]].sort_values("weighted_vote", ascending=False)[:10]

,title,vote_average,weighted_vote,vote_count
1881,The Shawshank Redemption,8.5,8.396052,8205
3337,The Godfather,8.4,8.263591,5893
662,Fight Club,8.3,8.216455,9413
3232,Pulp Fiction,8.3,8.207102,8428
65,The Dark Knight,8.2,8.136930,12002
1818,Schindler's List,8.3,8.126069,4329
3865,Whiplash,8.3,8.123248,4254
809,Forrest Gump,8.2,8.105954,7927
2294,Spirited Away,8.3,8.105867,3840
2731,The Godfather: Part II,8.3,8.079586,3338


In [127]:
def find_sim_movie(df,sorted_ind, title_name, top_n=10):
    title_movie = df[df["title"]==title_name]
    title_index = title_movie.index.values
    
    # top_n의 2배에 해당하는 장르 유사성이 높은 인덱스 추출
    similar_indexes = sorted_ind[title_index, :(top_n*2)]
    similar_indexes = similar_indexes.reshape(-1)
    # 기준 영화 인덱스는 제외
    similar_indexes = similar_indexes[similar_indexes != title_index]
    
    # top_n의 2배에 해당하는 후보군에서 weighted_vote가 높은 순으로 top_n만큼 추출
    return df.iloc[similar_indexes].sort_values("weighted_vote", ascending=False)[:top_n]

In [128]:
similar_movies = find_sim_movie(movies_df, genre_sim_sorted_ind, "The Godfather", 10)
similar_movies[["title","vote_average","weighted_vote"]]

,title,vote_average,weighted_vote
1881,The Shawshank Redemption,8.5,8.396052
2731,The Godfather: Part II,8.3,8.079586
1663,Once Upon a Time in America,8.2,7.657811
3887,Trainspotting,7.8,7.591009
892,Casino,7.8,7.423040
281,American Gangster,7.4,7.141396
4041,This Is England,7.4,6.739664
1149,American Hustle,6.8,6.717525
2839,Rounders,6.9,6.530427
1370,21,6.5,6.413490
